### All Imports

In [1]:
from General.args import get_args
import General.utils as GU
from PD_var_ILP.run_var_ilp import run_var_ilp
from PD_mul_ILP.run_mul_ilp import run_mul_ilp
from PD_mul_greedy.run_mul_greedy import run_mul_greedy
from PD_single_LPath.run_pd_single_LPath import run_longest_path

### Set up gurobi optimization library


First, create a file called gurobi.json with the details of your gurobi license in the main directory. The file should be in the following format:

```json
{
  "WLSACCESSID": "XXXXX",
  "WLSSECRET": "XXXXX",
  "LICENSEID": 12345
}
```


### Global program cmd arguments

Set arguments to their default values

In [2]:
args = get_args()

args.output = 'Temp'

### Define global variables for upstream and downstream regions

In [3]:
cfg = GU.load_config("config.json")

### Run PD-Single-LPath

#### Define sequence
First, we will define our mutreg region sequence, we will take the first 100 nucleotides of the coding sequence of the CXDAR human protein gene as a small test dataset.

In [4]:
protein_name = 'CXDAR'
mutreg_nt = 'ATGGCGCTCCTGCTGTGCTTCGTGCTCCTGTGCGGAGTAGTGGATTTCGCCAGAAGTTTGAGTATCACTACTCCTGAAGAGATGATTGAAAAAGCCAAAGCTACTCCTGAAGAGATGATTGAAAAAGCCAAAG'

## Add upstream and downstream sequences
full_sequence = cfg.upstream + mutreg_nt + cfg.downstream

#### Run longest path algorithm

We will runs PD-Single_LPath which runs a longest path algorithms on the primer graph, which is a DAG, to find the most efficient primers.

In [5]:
run_df, primer_paths = run_longest_path(full_sequence, mutreg_nt, protein_name, args, cfg) 

print("Results DataFrame:")
run_df

Saved selected primers to: Temp/PD_single_LPath_selected_primers.csv
Results DataFrame:


,protein_name,graph_nodes,graph_edges,graph_time_sec,longest_path_efficiency,total_time_sec,num_primers
0,CXDAR,3408,236389,2.949,1.61325,8.372,4


#### Selected primer path and primer efficiency

Print all selected primers in longest path and their efficiency

In [6]:
print("Primer Paths:")
for path in primer_paths:
    print(path)

print("Primer effiency")
print(run_df['longest_path_efficiency'].prod())   

Primer Paths:
(-58, -34, 'f')
(119, 141, 'r')
(93, 115, 'f')
(277, 296, 'r')
Primer effiency
1.6132498038728875


### Run with Tm & GC content thresholds

Apply thresholds on primer melting temperatures (tm), GC content, and maximum difference between fwd and reverse primers 

In [7]:
args.apply_threshold = True

args.min_tm = 58
args.max_tm= 65

args.min_tm = 40
args.max_tm= 60

args.max_difference = 3

run_df, primer_paths = run_longest_path(full_sequence, mutreg_nt, protein_name, args, cfg) 

print("Primer Paths:")
for path in primer_paths:
    print(path)
    
print(run_df['longest_path_efficiency'].prod())   

args.apply_threshold = False

Saved selected primers to: Temp/PD_single_LPath_selected_primers.csv
Primer Paths:
(-58, -34, 'f')
(123, 143, 'r')
(94, 118, 'f')
(280, 299, 'r')
1.1101513634330775


### Run PD-var-ILP

We will run PD-var-ILP on multiple variants of the same protein.

For this part, we will use the same sequence of the CXDAR protein as PD-single-LPath.
We will use the default number of variants, which is 3, and the deafult maximum overlap length, which is 6. 

In [8]:
results_df, ilp_path, greedy_path  = run_var_ilp(full_sequence, mutreg_nt, protein_name,args,cfg)
print("Variable ILP Results DataFrame:")
results_df

Running greedy algorithm
Running ILP
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2499075
Academic license 2499075 - for non-commercial use only - registered to jo___@live.biu.ac.il
Number of Constraints: 7104
Average Vars Per Constraint 84.39386261261261
Graph edges:  236389
Finished ILP Variable Creations
0
3
7
11
15
19
23
27
31
35
39
43
47
51
55
59
63
67
71
75
79
83
87
91
95
99
Finished Intersection constraints!
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Rocky Linux 9.7 (Blue Onyx)")

CPU model: Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads

Academic license 2499075 - for non-commercial use only - registered to jo___@live.biu.ac.il
Optimize a model with 10512 rows, 236389 columns and 40624023 nonzeros
Model fingerprint: 0x309eda85
Variable types: 0 continuous, 236389 integer (236389 binary)
Coefficient statistics:


,protein_name,num_variants,seq_length,graph_nodes,graph_edges,graph_time_sec,greedy_path_length,ilp_num_vars,ilp_path_length,ilp_num_constraints,ilp_setup_time_sec,ilp_optimize_time_sec,ILP_primer_efficiency,ILP_solution_feasibility,greedy_solution_feasibility,greedy_primer_efficiency,greedy_time_sec
0,CXDAR,3,133,3408,236389,2.945,10,236389,10,10512,92.367,101.068,2.526991,FEASIBLE,FEASIBLE,2.526991,1.194


#### Compare PD-Var-ILP with greedy baseline

We will compare the primers found by the ILP and by the greedy approach and their total efficiencies.

In [9]:
print("ILP chosen primers:")

for i,path in enumerate(ilp_path):
    print("Variant", i+1)
    print(path)

print("ILP primer efficiency: ",results_df['ILP_primer_efficiency'].prod())


print("Greedy chosen primers:")
for i,path in enumerate(greedy_path):
    print("Variant", i+1)
    print(path)

print("Greedy primer efficiency: ",results_df['greedy_primer_efficiency'].prod())

ILP chosen primers:
Variant 1
["(-81, -60, 'f')", "(92, 114, 'r')", "(64, 89, 'f')", "(241, 259, 'r')"]
Variant 2
["(-58, -34, 'f')", "(119, 141, 'r')", "(93, 115, 'f')", "(277, 296, 'r')"]
Variant 3
["(-31, -1, 'f')", "(156, 174, 'r')"]
ILP primer efficiency:  2.5269911115148025
Greedy chosen primers:
Variant 1
[(-58, -34, 'f'), (119, 141, 'r'), (93, 115, 'f'), (277, 296, 'r')]
Variant 2
[(-81, -60, 'f'), (92, 114, 'r'), (64, 89, 'f'), (241, 259, 'r')]
Variant 3
[(-31, -1, 'f'), (156, 174, 'r')]
Greedy primer efficiency:  2.526991111514803


#### Run PD-Var_ILP with merging bins corresponding to identical sequences

Run using feature which imposes ILP constraints that also merge bins with identical sequence

In [10]:
args.merge_bins = True
results_df, ilp_path, greedy_path  = run_var_ilp(full_sequence, mutreg_nt, protein_name,args,cfg)
print("Variable ILP Results DataFrame:")
results_df

Running greedy algorithm
Running ILP
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2499075
Academic license 2499075 - for non-commercial use only - registered to jo___@live.biu.ac.il
Number of Constraints: 7104
Average Vars Per Constraint 84.39386261261261
Number of united bins constraints: 6596
Average Vars Per Constraint 90.89357186173439
Graph edges:  236389
Finished ILP Variable Creations
0
3
7
11
15
19
23
27
31
35
39
43
47
51
55
59
63
67
71
75
79
83
87
91
95
99
Finished Intersection constraints!
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Rocky Linux 9.7 (Blue Onyx)")

CPU model: Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads

Academic license 2499075 - for non-commercial use only - registered to jo___@live.biu.ac.il
Optimize a model with 10004 rows, 236389 columns and 40621943 nonzeros
Model fingerprint: 0x8e2dbaf0

,protein_name,num_variants,seq_length,graph_nodes,graph_edges,graph_time_sec,greedy_path_length,ilp_num_vars,ilp_path_length,ilp_num_constraints,ilp_setup_time_sec,ilp_optimize_time_sec,ILP_primer_efficiency,ILP_solution_feasibility,greedy_solution_feasibility,greedy_primer_efficiency,greedy_time_sec
0,CXDAR,3,133,3408,236389,3.029,10,236389,10,10004,88.012,98.115,0.933002,FEASIBLE,FEASIBLE,2.526991,1.067


### Running PD-mul-ILP

We will run PD-mul-ILP on multiple non-homologous proteins. For this part, we will add another partial 100-nt sequence of the SHP2 protein

In [11]:
protein_name1 = 'CXDAR'
mutreg_nt1 = 'ATGGCGCTCCTGCTGTGCTTCGTGCTCCTGTGCGGAGTAGTGGATTTCGCCAGAAGTTTGAGTATCACTACTCCTGAAGAGATGATTGAAAAAGCCAAAG'
## Add upstream and downstream sequences
full_sequence1 = cfg.upstream + mutreg_nt1 + cfg.downstream

protein_name2 = 'SHP2'
mutreg_nt2 = 'ATGACATCGCGGAGATGGTTTCACCCAAATATCACTGGTGTGGAGGCAGAAAACCTACTGTTGACAAGAGGAGTTGATGGCAGTTTTTTGGCAAGGCCTA'
full_sequence2 = cfg.upstream + mutreg_nt2 + cfg.downstream

## create protein dataset
mutreg_regions = [mutreg_nt1, mutreg_nt2]
sequences_nt = [full_sequence1, full_sequence2]
protein_names = [protein_name1, protein_name2]


#### Run PD-mul-ILP 2 non-homolous proteins parts
PD-mul-ILP uses an integer linear program with forbidden pair constraints on all cross-hybridizing pairs.

In [12]:

run_df, primer_paths = run_mul_ilp(mutreg_regions, sequences_nt, protein_names, args, cfg)

print("PD-mul_ILP Results DataFrame:")
run_df

[STEP] Creating graphs for 2 proteins...
Creating Graph for protein:  0 Protein name:  CXDAR
Creating Graph for protein:  1 Protein name:  SHP2
[DONE] Graphs created in 19.00 sec (peak 133.5 MB).
[STEP] Finding forbidden pairs across proteins...
Finding intra-protein forbidden pairs constraints
[1/2] Processing CXDAR...


[2/2] Processing SHP2...
Number of intra-protein forbidden pairs constraints:  0
Finding inter-protein forbidden pairs for 1 combinations...
[1/1] Fractions of pairs considered
Number of inter-protein forbidden pairs constraints:  0
[DONE] Forbidden pairs in 24.56 sec (intra=0, inter=0).
[STEP] Running ILP model...
Creating ILP model...
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2499075
Academic license 2499075 - for non-commercial use only - registered to jo___@live.biu.ac.il
Adding constraints to ILP model...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Rocky Linux 9.7 (Blue Onyx)")

CPU model: Intel(R) Xeon(R) Gold 6338 CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads

Academic license 2499075 - for non-commercial use only - registered to jo___@live.biu.ac.il
Optimize a model with 5100 rows, 321132 columns and 642264 nonzeros
Model fingerprint:

,num_proteins,graph_time_sec,ilp_num_vars,ilp_num_constraints,ilp_intra_forbidden_cnt,ilp_inter_forbidden_cnt,forbidden_time_sec,ilp_setup_time_sec,ilp_optimize_time_sec,ilp_feasibility,total_primer_efficiency,num_primers
0,2,19,321132,5100,0,0,24.560111,12.352175,1.121767,FEASIBLE,3.375891,8


#### Primer slection and total efficiency

We will look at the primer paths found for each protein by PD-mul-ILP, and the total efficiency of the selected primers.

In [13]:
print("Chosen primers:")

for protein, path in primer_paths.items():
    print("Protein:", protein)
    print(path)

total_efficiency = run_df['total_primer_efficiency'].prod()   
print("Total primer efficiency:")
print(total_efficiency) 

Chosen primers:
Protein: CXDAR
["(-80, -60, 'f')", "(96, 115, 'r')", "(70, 96, 'f')", "(244, 265, 'r')"]
Protein: SHP2
["(-80, -60, 'f')", "(98, 118, 'r')", "(69, 92, 'f')", "(244, 264, 'r')"]
Total primer efficiency:
3.3758909432490496


### Testing behavior with infeasible primer sequences

We test a case where no feasible primer set exists. Specifically, two sequences are constructed such that their upstream regions are reverse complements, creating primer pairs with high melting temperature and cross-hybridization risk.

These primer pairs are forbidden by the ILP constraints, and therefore PrimerDesigner correctly reports that no feasible solution exists.

In [14]:
cfg_mod = GU.Config(
    upstream='ATGGCGCTCCTGCTGTGC',
    downstream=cfg.downstream,
    max_tm=cfg.max_tm
)

args.primer_lmax = 18

protein_name1 = 'CXDAR'
mutreg_nt1 = 'ATGGCGCTCCTGCTGTGCTTCGTGCTCCTGTGCGGAGTAGTGGATTTCGCCAGAAGTTTGAGTATCACTACTCCTGAAGAGATGATTGAAAAAGCCAAAG'
## Add upstream and downstream sequences
full_sequence1 = cfg_mod.upstream + mutreg_nt1 + cfg.downstream

## Add reverse complement upstream sequence for the other protein
protein_name2 = 'SHP2'
mutreg_nt2 = 'ATGACATCGCGGAGATGGTTTCACCCAAATATCACTGGTGTGGAGGCAGAAAACCTACTGTTGACAAGAGGAGTTGATGGCAGTTTTTTGGCAAGGCCTA'
# full_sequence2 = cfg_mod.upstream + mutreg_nt2 + cfg.downstream
full_sequence2 = GU.revcomp(cfg_mod.upstream) + mutreg_nt2 + cfg.downstream

## create protein dataset
mutreg_regions = [mutreg_nt1, mutreg_nt2]
sequences_nt = [full_sequence1, full_sequence2]
protein_names = [protein_name1, protein_name2]


In [ ]:
run_df, primer_paths = run_mul_ilp(mutreg_regions, sequences_nt, protein_names, args, cfg_mod)

print("PD-mul_ILP Results DataFrame:")
run_df

## reseet primer_lmax
args.primer_lmax = 30

[STEP] Creating graphs for 2 proteins...
Creating Graph for protein:  0 Protein name:  CXDAR
Creating Graph for protein:  1 Protein name:  SHP2
[DONE] Graphs created in 0.00 sec (peak 1.6 MB).
[STEP] Finding forbidden pairs across proteins...
Finding intra-protein forbidden pairs constraints
[1/2] Processing CXDAR...
[2/2] Processing SHP2...
Number of intra-protein forbidden pairs constraints:  0
Finding inter-protein forbidden pairs for 1 combinations...
[1/1] Fractions of pairs considered
Number of inter-protein forbidden pairs constraints:  100
[DONE] Forbidden pairs in 7.50 sec (intra=0, inter=100).
[STEP] Running ILP model...
Creating ILP model...
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2499075
Academic license 2499075 - for non-commercial use only - registered to jo___@live.biu.ac.il
Adding constraints to ILP model...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Rocky Linux 9.7 (Blue Onyx)")

CPU model: Intel(R) Xeon(R) G

### Running PD-mul-Greedy

We will run PD-mul-Greedy, which uses a greedy algorithm to find efficient primers paths for each protein, on multiple non-homologous proteins. 
We will use the same proteins as we used for the PD-mul-ILP run. 

In [ ]:
run_df, paths = run_mul_greedy(sequences_nt,mutreg_regions, protein_names, args, cfg)

print("PD-mul_greedy Results DataFrame:")
run_df

[INFO] Processing protein 0/2: CXDAR
[SAVE] Selected primers CSV → Temp/PD_mul_Greedy_selected_primers.csv
PD-mul_greedy Results DataFrame:


{'num_proteins': 2,
 'total_primer_efficiency': 2.329953557394205,
 'greedy_time_sec': 10.817,
 'cross_hybridizations_cnt': 0,
 'proteins_with_reiterations_cnt': 0,
 'total_reiterations': 0,
 'unresolved_proteins_cnt': 0,
 'unresolved_proteins': '',
 'total_primers': 4}

#### Primers found for each protein total efficiency

We will examine the total efficiency and the primers found for each of the proteins by the greedy algorithm

In [17]:
for protein, path in paths.items():
    print("Protein:", protein)
    print(path)

total_efficiency = run_df['total_primer_efficiency']   
print("Total primer efficiency:")
print(total_efficiency)


Protein: CXDAR
[(-51, -33, 'f'), (125, 146, 'r')]
Protein: SHP2
[(-74, -54, 'f'), (113, 131, 'r')]
Total primer efficiency:
2.329953557394205
